## Conversation with the sql database

In [ ]:
import os
os.environ['OPENAI_API_KEY']

In [ ]:
#testing DB
import sqlite3
conn = sqlite3.connect('Chinook.db')
cursor = conn.cursor()
cursor.execute('SELECT * FROM Artist LIMIT 10')
rows = cursor.fetchall()
for row in rows:
    print(row)
cursor.close()
conn.close()

(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'AntÃ´nio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor, AgentType
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

In [ ]:
# Create the agent executor
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [ ]:
db.get_table_info()

c:\Users\srika\anaconda3\lib\site-packages\langchain\sql_database.py:353: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore


'\nCREATE TABLE "Album" (\n\t"AlbumId" INTEGER NOT NULL, \n\t"Title" NVARCHAR(160) NOT NULL, \n\t"ArtistId" INTEGER NOT NULL, \n\tPRIMARY KEY ("AlbumId"), \n\tFOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")\n)\n\n/*\n3 rows from Album table:\nAlbumId\tTitle\tArtistId\n1\tFor Those About To Rock We Salute You\t1\n2\tBalls to the Wall\t2\n3\tRestless and Wild\t2\n*/\n\n\nCREATE TABLE "Artist" (\n\t"ArtistId" INTEGER NOT NULL, \n\t"Name" NVARCHAR(120), \n\tPRIMARY KEY ("ArtistId")\n)\n\n/*\n3 rows from Artist table:\nArtistId\tName\n1\tAC/DC\n2\tAccept\n3\tAerosmith\n*/\n\n\nCREATE TABLE "Customer" (\n\t"CustomerId" INTEGER NOT NULL, \n\t"FirstName" NVARCHAR(40) NOT NULL, \n\t"LastName" NVARCHAR(20) NOT NULL, \n\t"Company" NVARCHAR(80), \n\t"Address" NVARCHAR(70), \n\t"City" NVARCHAR(40), \n\t"State" NVARCHAR(40), \n\t"Country" NVARCHAR(40), \n\t"PostalCode" NVARCHAR(10), \n\t"Phone" NVARCHAR(24), \n\t"Fax" NVARCHAR(24), \n\t"Email" NVARCHAR(60) NOT NULL, \n\t"SupportRepId" INTEG

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

In [ ]:
agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:
agent_executor.run("Describe the playlisttrack table")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: The PlaylistTrack table is the most relevant to the question.
Action: sql_db_schema
Action Input: PlaylistTrack
Observation: 
CREATE TABLE "PlaylistTrack" (
	"PlaylistId" INTEGER NOT NULL, 
	"TrackId" INTEGER NOT NULL, 
	PRIMARY KEY ("PlaylistId", "TrackId"), 
	FOREIGN KEY("TrackId") REFERENCES "Track" ("TrackId"), 
	FOREIGN KEY("PlaylistId") REFERENCES "Playlist" ("PlaylistId")
)

/*
3 rows from PlaylistTrack table:
PlaylistId	TrackId
1	3402
1	3389
1	3390
*/
Thought: I now know the final answer
Final Answer: The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.

> Finished chain.


'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

Answer:- 'The PlaylistTrack table contains two columns, PlaylistId and TrackId, which are both integers and form a primary key. It also has two foreign keys, one to the Track table and one to the Playlist table.'

In [ ]:
agent_executor.run("concat the artist table and album table and get top 5 artists name and their album")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Artist and Album tables
Action: sql_db_schema
Action Input: Artist, Album
Observation: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Thought: I should query the Artist and Album tables to get the top 5 artists name and their album
Action: sql_db_query
Action Input: SELECT Artist.Name

'AC/DC - For Those About To Rock We Salute You, AC/DC - Let There Be Rock, Aaron Copland & London Symphony Orchestra - A Copland Celebration, Vol. I, Aaron Goldberg - Worlds, Academy of St. Martin in the Fields & Sir Neville Marriner - The World of Classical Favourites'

In [ ]:
agent_executor.run("How many employees are there?")



> Entering new  chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Employee table.
Action: sql_db_schema
Action Input: Employee
Observation: 
CREATE TABLE "Employee" (
	"EmployeeId" INTEGER NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"FirstName" NVARCHAR(20) NOT NULL, 
	"Title" NVARCHAR(30), 
	"ReportsTo" INTEGER, 
	"BirthDate" DATETIME, 
	"HireDate" DATETIME, 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60), 
	PRIMARY KEY ("EmployeeId"), 
	FOREIGN KEY("ReportsTo") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Employee table:
EmployeeId	LastName	FirstName	Title	ReportsTo	BirthDate	HireDate	Address	City	State	Country	PostalCode	Phone	Fax	Email
1	Adams	Andrew	General Ma

'There are 8 employees.'

Answer:- 'There are 8 employees.'

### Using Query Checker
- Language Model generates invalid SQL with small mistakes sometimes that can be self-corrected using the same technique used by the SQL Database Agent to try and fix the SQL using the LLM.

In [ ]:
llm = OpenAI(temperature=0, verbose=True)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, use_query_checker=True)

In [ ]:
db_chain.run("How many albums by Aerosmith?")



> Entering new  chain...
How many albums by Aerosmith?
SQLQuery:SELECT COUNT(*) FROM Album INNER JOIN Artist ON Album.ArtistId = Artist.ArtistId WHERE Artist.Name = 'Aerosmith';
SQLResult: [(1,)]
Answer:There is 1 album by Aerosmith.
> Finished chain.


'There is 1 album by Aerosmith.'

In [ ]:
db_chain.run("What are some example tracks by Bach?")



> Entering new  chain...
What are some example tracks by Bach?
SQLQuery:SELECT "Name" FROM "Track" WHERE "Composer" LIKE '%Bach%' LIMIT 5;
SQLResult: [('American Woman',), ('Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace',), ('Aria Mit 30 VerÃ¤nderungen, BWV 988 "Goldberg Variations": Aria',), ('Suite for Solo Cello No. 1 in G Major, BWV 1007: I. PrÃ©lude',), ('Toccata and Fugue in D Minor, BWV 565: I. Toccata',)]
Answer:Some example tracks by Bach are American Woman, Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace, Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria, Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude, and Toccata and Fugue in D Minor, BWV 565: I. Toccata.
> Finished chain.


'Some example tracks by Bach are American Woman, Concerto for 2 Violins in D Minor, BWV 1043: I. Vivace, Aria Mit 30 Veränderungen, BWV 988 "Goldberg Variations": Aria, Suite for Solo Cello No. 1 in G Major, BWV 1007: I. Prélude, and Toccata and Fugue in D Minor, BWV 565: I. Toccata.'

## Chat with text data

In [ ]:
from PyPDF2 import PdfReader
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain

In [ ]:
load_dotenv()
# upload file
pdf = r"C:\Users\srika\langchain-learning\chat\state_of_the_union.pdf"

# extract the text
pdf_reader = PdfReader(pdf)
text = ""
for page in pdf_reader.pages:
    text += page.extract_text()
text

"Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress a\nnd the Cabinet. Justices of the Supreme Court. My fellow Americans.   \n \nLast year COVID-19 kept us apart. This year we are finally together again.  \n \nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans.  \n \nWith a duty to one another to the American people to the Constitution.  \n \nAnd with an unwavering resolve that freedom will always triumph over tyranny.  \n \nSix days ago, Russia's Vladimir Putin sought to shake the foundations of the free world thinking he could \nmake it bend to his menacing ways. But he badly miscalculated.  \n \nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he n\never imagined.  \n \nHe met the Ukrainian people.  \n \nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspir\nes the world.  \n \nGroups of cit

In [ ]:
# split into chunks
text_splitter = CharacterTextSplitter(
separator="\n",
chunk_size=1000,
chunk_overlap=200,
length_function=len
)
chunks = text_splitter.split_text(text)

# create embeddings
embeddings = OpenAIEmbeddings()
knowledge_base = FAISS.from_texts(chunks, embeddings)

In [ ]:
# show user input
user_question = input("Ask a question about your PDF:")
if user_question:
    docs = knowledge_base.similarity_search(user_question)

llm = OpenAI()
chain = load_qa_chain(llm, chain_type="stuff")
with get_openai_callback() as cb:
    response = chain.run(input_documents=docs, question=user_question)
    # We can see the additional information
    print(cb)

Tokens Used: 1005
	Prompt Tokens: 981
	Completion Tokens: 24
Successful Requests: 1
Total Cost (USD): $0.0201


### Q-What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?

In [ ]:
response

' The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.'

### Chat with the text files

In [ ]:
!pip install chromadb

In [ ]:
import json

with open('./state_of_the_union.json') as f:
  dataset = json.load(f)

In [ ]:
from langchain.document_loaders import TextLoader
from langchain.indexes import VectorstoreIndexCreator

loader = TextLoader("./state_of_the_union.txt")
vectorstore = VectorstoreIndexCreator().from_loaders([loader]).vectorstore

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    chain_type="stuff",
    retriever=vectorstore.as_retriever(),
    input_key="question",
)

In [ ]:
chain(dataset[0])

{'question': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
 'result': ' The NATO Alliance was created to secure peace and stability in Europe after World War 2.'}

In [ ]:
chain("What did the president say about Justice Breyer?")

{'question': 'What did the president say about Justice Breyer/',
 'result': ' The President said that Justice Breyer has dedicated his life to serve this country and thanked him for his service.'}

### For chatbot, for writing SQL query
- price factor
- accuracy